In [ ]:
def aspect_pol(negatif,positif,text):
    #text = laptop_train[0]
    token = nltk.word_tokenize((text[0].lower()))
    tagged = nltk.pos_tag(token)
    termslist = []
    res = []
    for i in range(len(text)-1):
        termslist.append(text[i+1]['term'])
    terms = termslist.copy() 
    for word in token :
        for aspect in termslist:
            if word in aspect :
                part = text[0][:text[0].find(aspect)] # pas ici ajouter fonction qui recup les mots clés
                res.append(before_neg_matching(negatif, positif, part))
                termslist.remove(aspect)
    return terms, res

In [ ]:
#aspect_pol(negatif,positif,resto_train[2])

In [ ]:
#Polarité par apsect et par com (ancienne version)
def com_polarity(negatif,positif,data):
    datapol = []
    for i in range(len(data)):
        #datapol.append(before_neg_matching(negatif,positif,data[i][0])) 
        datapol.append(aspect_pol(negatif,positif,data[i])) 
    return datapol

In [ ]:
#Polarité par apsect et par com (ancienne version)
resfinal_resto = com_polarity(negatif,positif,resto_train)
resfinal_laptop = com_polarity(negatif,positif,laptop_train)

In [ ]:
#ON compare les mots avec la base de données (ancienne version)
def is_matching(negatif,positif,texte):
    nbneg = 0
    nbpos = 0
    #texte = no_stopword(texte)
    for i in range(len(texte)):
        for mot1 in negatif:
            if texte[i].lower() == mot1[0].lower():
                nbneg += 1            
        for mot2 in positif :
            if texte[i].lower() == mot2[0].lower():
                nbpos += 1
    return nbneg/len(texte), nbpos/len(texte)

In [ ]:
#def sentiword(com):
com =  resto_keywords[0][0].copy()
print(com[0][1])
for i in range(len(com)):
    if com[i][1] == "RB" :
        print((list(swn.senti_synsets(com[i][0],'r'))[0]).neg_score())
(list(swn.senti_synsets("die",'r'))[0]).neg_score()

In [ ]:
#Polarité par apsect en prenant en compte les mots neg et par com (ancienne version)
def before_neg_matching(negatif, positif, text):
    nbneg = 0
    nbpos = 0
    texte = nltk.word_tokenize((text.lower()))
    if "not" in texte or "n't" in texte:
        for mot1 in negatif:
            if texte[0].lower() == mot1[0].lower():
                nbneg += 1            
        for mot2 in positif :
            if texte[0].lower() == mot2[0].lower():
                nbpos += 1
        for i in range(len(texte)-1):
            for mot1 in negatif:
                if texte[i+1].lower() == mot1[0].lower():
                    if texte[i] == "not" or texte[i] =="n't":
                        nbpos += 1
                    else :
                        nbneg += 1
            for mot2 in positif :
                if texte[i+1].lower() == mot2[0].lower():
                    if texte[i] == "not" or texte[i] =="n't":
                        nbneg += 1
                    else :
                        nbpos += 1
    else :
        for i in range(len(texte)):
            for mot1 in negatif:
                if texte[i].lower() == mot1[0].lower():
                    nbneg += 1            
            for mot2 in positif :
                if texte[i].lower() == mot2[0].lower():
                    nbpos += 1
    #return [nbneg/len(texte), nbpos/len(texte)]
    return [nbneg, nbpos]

In [ ]:
def com_polarity(data):
    datalist = []
    for com in data :
        comlist =  []
        for aspect in com:
            aspectlist = []
            for tagged in aspect :
                '''
                type =""
                if tagged[1] == "NN":
                    type = 'n'
                elif tagged[1] == "RP" or tagged[1] == "RB":
                    type = 'r'                    
                elif tagged[1] == "JJ":
                    type = 'a'
                '''  
                if len(tagged) != 0 :
                    word = lem.lemmatize(tagged[0])
                else :
                    word = " "
                nbneg = 0
                nbpos = 0
                for mot_neg in negatif:
                    ratio = difflib.SequenceMatcher(None,word,mot_neg[0]).ratio()
                    if ratio > 0.95:
                        nbneg += 1
                for mot_pos in positif:
                    ratio = difflib.SequenceMatcher(None,word,mot_pos[0]).ratio()
                    if ratio > 0.95:
                        nbpos += 1       
                if nbneg>nbpos :
                    aspectlist.append(-1)
                elif nbneg<nbpos:
                    aspectlist.append(1)
                elif nbneg == nbpos:
                    aspectlist.append(0)
            if len(aspectlist) < 1 :
                aspectlist.append(0)
            somme = sum(aspectlist)
            if somme > 0 :
                comlist.append(1)
            elif somme < 0 :
                comlist.append(-1)
            else :               
                comlist.append(0)
                
        datalist.append(comlist)
    return datalist 